<a href="https://colab.research.google.com/github/Lorddickenstein/FSLRwithNLP/blob/main/Application/CNN_Model_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network with Own Datasets

In [2]:
from google.colab import files

from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Activation, Dense, Flatten, MaxPool2D, Conv2D, Dropout, BatchNormalization
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2
import glob
import warnings
import random
import shutil
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

If using a GPU, run this to make the computer know that you have a gpu so tensorflow could identify it correctly and enable memory growth on the gpu.

Otherwise, leave this code.

In [ ]:
physical_devices = tf.config.expertimental.list_physical_devices('GPU')
print('Nump GPUs Available: ', len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Data Preparation
The images are found inside the OurDataset Folder. The child folder named Old_Dataset will be used as an example. It will be read by the program and the output will be put into the Old_Dataset_Preprocessed.

In [4]:
# Organize data into train, valid, test dirs
folders = ['train', 'valid', 'test']
dataset_categories = ['Dynamic_Single', 'Dynamic_Double',
                       'Static_Single', 'Static_Double',
                       'Letters', 'Numbers']
letters = ['A', 'B', 'C', 'D', 'E',
           'F', 'G', 'H', 'I', 'J',
           'K', 'L', 'M', 'N', 'O',
           'P', 'Q', 'R', 'S', 'T',
           'U', 'V', 'W', 'X', 'Y',
           'Z']
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
static_single = []
static_double = []
dynamic_single = []
dynamic_double = []

train_size = 50
valid_size = 20
test_size = 10

dataset_classes = [numbers, letters,
                   static_single, static_double,
                   dynamic_single, dynamic_double]

In [ ]:
# Create a directory from for all classes in class_obj
def make_dir(path, class_obj):
  for obj in class_obj:
    path_class = os.path.join(path, obj)
    os.makedirs(path_class)

In [6]:
# Access all images from directory
root = '/content/drive/MyDrive/Colab Notebooks/Datasets/OurDataset'
train_path = os.path.join(root, "Train")
valid_path = os.path.join(root, "Valid")
test_path = os.path.join(root, "Test")

# Make class directories for Train, Valid, and Test Folders
if os.path.isdir(root + '/Train') is False:
  make_dir(train_path, letters)
  make_dir(train_path, numbers)
  make_dir(train_path, static_single)
  make_dir(train_path, static_double)
  make_dir(train_path, dynamic_single)
  make_dir(train_path, dynamic_double)
  
  make_dir(valid_path, letters)
  make_dir(valid_path, numbers)
  make_dir(valid_path, static_single)
  make_dir(valid_path, static_double)
  make_dir(valid_path, dynamic_single)
  make_dir(valid_path, dynamic_double)

  make_dir(test_path, letters)
  make_dir(test_path, numbers)
  make_dir(test_path, static_single)
  make_dir(test_path, static_double)
  make_dir(test_path, dynamic_single)
  make_dir(test_path, dynamic_double)
else:
  print("Directory /Train already exists.")
  

Directory /Train already exists.


# Populate **Train**, **Valid**, and **Test** Folders
Populate the **Train, Valid, and Test** folder with directories of classes. Uses ***random sampling*** to randomize selection of images and uses ***glob*** to select file paths matching specific pattern in their names. ***shutil*** module allows the transfer of these files to the *Train, Valid, and Test*

In [ ]:
# Train Folder

# Dynamic Single
path = os.path.join(root, 'Old_Dataset/Dynamic_Single')
for class_obj in dynamic_single:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), train_size):
    shutil.move(item, train_path + '/' + class_obj)

# Dynamic Double
path = os.path.join(root, 'Old_Dataset/Dynamic_Double')
for class_obj in dynamic_double:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), train_size):
    shutil.move(items, train_path + '/' + class_obj)

# Static Single
path = os.path.join(root, 'Old_Dataset/Static_Single')
for class_obj in static_single:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), train_size):
    shutil.move(item, train_path + '/' + class_obj)

# Static Double
path = os.path.join(root, 'Old_Dataset/Static_Double')
for class_obj in static_double:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), train_size):
    shutil.move(items, train_path + '/' + class_obj)
  
# Letters
path = os.path.join(root, 'Old_Dataset/Letters')
for letter in letters:
  path_class = os.path.join(path, letter)
  os.chdir(path_class)
  for item in random.sample(glob.glob(letter + '_*'), train_size):
    shutil.move(item, train_path + '/' + letter)

# Numbers
path = os.path.join(root, 'Old_Dataset/Numbers')
for number in numbers:
  path_class = os.path.join(path, number)
  os.chdir(path_class)
  for item in random.sample(glob.glob(number + '_*'), train_size):
    shutil.move(item, train_path + '/' + number)

In [ ]:
# Valid Folder

# Dynamic Single
path = os.path.join(root, 'Old_Dataset/Dynamic_Single')
for class_obj in dynamic_single:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), valid_size):
    shutil.move(item, valid_path + '/' + class_obj)

# Dynamic Double
path = os.path.join(root, 'Old_Dataset/Dynamic_Double')
for class_obj in dynamic_double:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), valid_size):
    shutil.move(items, valid_path + '/' + class_obj)

# Static Single
path = os.path.join(root, 'Old_Dataset/Static_Single')
for class_obj in static_single:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), valid_size):
    shutil.move(item, valid_path + '/' + class_obj)

# Static Double
path = os.path.join(root, 'Old_Dataset/Static_Double')
for class_obj in static_double:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), valid_size):
    shutil.move(items, valid_path + '/' + class_obj)
  
# Letters
path = os.path.join(root, 'Old_Dataset/Letters')
for letter in letters:
  path_class = os.path.join(path, letter)
  os.chdir(path_class)
  for item in random.sample(glob.glob(letter + '*'), valid_size):
    shutil.move(item, valid_path + '/' + letter)

# Numbers
path = os.path.join(root, 'Old_Dataset/Numbers')
for number in numbers:
  path_class = os.path.join(path, number)
  os.chdir(path_class)
  for item in random.sample(glob.glob(number + '_*'), valid_size):
    shutil.move(item, valid_path + '/' + number)

In [ ]:
# Test

# Dynamic Single
path = os.path.join(root, 'Old_Dataset/Dynamic_Single')
for class_obj in dynamic_single:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), test_size):
    shutil.move(item, test_path + '/' + class_obj)

# Dynamic Double
path = os.path.join(root, 'Old_Dataset/Dynamic_Double')
for class_obj in dynamic_double:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), test_size):
    shutil.move(items, test_path + '/' + class_obj)

# Static Single
path = os.path.join(root, 'Old_Dataset/Static_Single')
for class_obj in static_single:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), test_size):
    shutil.move(item, test_path + '/' + class_obj)

# Static Double
path = os.path.join(root, 'Old_Dataset/Static_Double')
for class_obj in static_double:
  path_class = os.path.join(path, class_obj)
  os.chdir(path_class)
  for item in random.sample(glob.glob(class_obj + '_*'), test_size):
    shutil.move(items, test_path + '/' + class_obj)
  
# Letters
path = os.path.join(root, 'Old_Dataset/Letters')
for letter in letters:
  path_class = os.path.join(path, letter)
  os.chdir(path_class)
  for item in random.sample(glob.glob(letter + '_*'), test_size):
    shutil.move(item, test_path + '/' + letter)

# Numbers
path = os.path.join(root, 'Old_Dataset/Numbers')
for number in numbers:
  path_class = os.path.join(path, number)
  os.chdir(path_class)
  for item in random.sample(glob.glob(number + '_*'), test_size):
    shutil.move(item, test_path + '/' + number)

# Preprocess Image
Transform images from the dataset into a format that the model expect. Use the VGG16 preprocessing input.

In [ ]:
classes = letters + numbers + dynamic_single + dynamic_double + static_single + static_double
image_size = (224, 224)

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.application.vgg16.preprocess_input) \
    .flow_from_directory(director=train_path, target_size=image_size, classes=classes, batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.application.vgg16.preprocess_input) \
    .flow_from_directory(director=label_path, target_size=image_size, classes=classes, batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.application.vgg16.preprocess_input) \
    .flow_from_directory(director=test_path, target_size=image_size, classes=classes, batch_size=10)

In [ ]:
assert train_batches.n == 50
assert valid_batches.n == 20
assert test_batches.n == 10

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
# Plot 10 sample images
def plotImages(images_arr):
  fig, axes = plt.subplot(1, 10, figsize=(20, 20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
plotImages(img)
print(labels)

# Build and Train the model

In [8]:
# Count the total classes that the model must know
total_classes = len(os.listdir(train_path))
print(total_classes)

35


In [13]:
# Create the Sequential Model
model = Sequential()

# Add Layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(total_classes, activation='softmax'))

# Summary of layers
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)              

In [ ]:
# Create the connection and train the model
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(x=train_batches, validation_data=valid_batches, batch_size=15, epochs=30)

# Evaluate the model with test_sets
print(model.evaluate(test_batches))

In [ ]:
# save the model
model_name = 'FSLR_using_CNN.h5'
path = '/content/drive/MyDrive/Colab Notebooks/CNN Models/' + model_name
model.save(path)